In [47]:
# Import necessary libraries

import pandas as pd
import ast
import string
import pickle

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
# Import csv files

movies = pd.read_csv('Movies 10k.csv')
credit = pd.read_csv('Credit 10k.csv')
keywords = pd.read_csv('keywords.csv')
image = pd.read_csv('image_url.csv')

In [49]:
movies.shape,credit.shape, keywords.shape, image.shape

((10000, 14), (10000, 3), (10000, 2), (9180, 2))

In [39]:
movies.head(2)

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre
0,False,/cVh8Af7a9JMOJl75ML3Dg2QVEuq.jpg,762509,en,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...",3353.171,/9bXHaLlsFYpJUutg4E6WXAjaxDi.jpg,2024-12-18,Mufasa: The Lion King,False,7.462,1178,"['Adventure', 'Family', 'Animation']"
1,False,/zo8CIjJ2nfNOevqNajwMRO6Hwka.jpg,1241982,en,Moana 2,After receiving an unexpected call from her wa...,2296.395,/aLVkiINlIeCkcZIzb7XHzPYgO6L.jpg,2024-11-21,Moana 2,False,7.200,1605,"['Animation', 'Adventure', 'Family', 'Comedy']"


In [50]:
movies= movies[['id','title','overview','genre']]

In [70]:
movies.shape

(8472, 4)

In [64]:
movies.duplicated().sum()
movies.drop_duplicates(inplace = True)

In [72]:
# merge all the csv files into one dataframe
df = movies.merge(credit, on = 'id')
df = df.merge(keywords, on = 'id')
df = df.merge(image, on = 'id')

In [73]:
df.shape

(18002, 8)

In [74]:
df.head(2)

,id,title,overview,genre,cast,crew,keywords,image_url
0,762509,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...","['Adventure', 'Family', 'Animation']","[{'adult': False, 'gender': 2, 'id': 1763709, ...","[{'adult': False, 'gender': 2, 'id': 42265, 'k...","[{'id': 6054, 'name': 'friendship'}, {'id': 43...",http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUut...
1,1241982,Moana 2,After receiving an unexpected call from her wa...,"['Animation', 'Adventure', 'Family', 'Comedy']","[{'adult': False, 'gender': 1, 'id': 1564846, ...","[{'adult': False, 'gender': 2, 'id': 4524084, ...","[{'id': 658, 'name': 'sea'}, {'id': 270, 'name...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...


In [75]:
# check for null values and then dropping rows with null values
print(df.isnull().sum())
df.dropna(inplace = True)
print('\n')
# drop duplicate data while keeping the only original data
print(df.duplicated().sum())
df.drop_duplicates(inplace = True, keep = 'first')


id            0
title         0
overview     16
genre         0
cast          0
crew          0
keywords      0
image_url     0
dtype: int64


10206


In [76]:
df.shape

(7780, 8)

In [77]:
df.head(2)

,id,title,overview,genre,cast,crew,keywords,image_url
0,762509,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...","['Adventure', 'Family', 'Animation']","[{'adult': False, 'gender': 2, 'id': 1763709, ...","[{'adult': False, 'gender': 2, 'id': 42265, 'k...","[{'id': 6054, 'name': 'friendship'}, {'id': 43...",http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUut...
1,1241982,Moana 2,After receiving an unexpected call from her wa...,"['Animation', 'Adventure', 'Family', 'Comedy']","[{'adult': False, 'gender': 1, 'id': 1564846, ...","[{'adult': False, 'gender': 2, 'id': 4524084, ...","[{'id': 658, 'name': 'sea'}, {'id': 270, 'name...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...


In [78]:
type(df['genre'].iloc[0])

str

In [79]:
# since df['genre'] has data as str so to evaluate them we use ast.literal_eval
import ast
df['genre'] = df['genre'].apply(ast.literal_eval)

In [80]:
df['cast'].iloc[0]

"[{'adult': False, 'gender': 2, 'id': 1763709, 'known_for_department': 'Acting', 'name': 'Aaron Pierre', 'original_name': 'Aaron Pierre', 'popularity': 61.94, 'profile_path': '/hNwZWdT2KxKj1YLbipvtUhNjfAp.jpg', 'cast_id': 73, 'character': 'Mufasa (voice)', 'credit_id': '6784ce60bd793c03544ec5ff', 'order': 0}, {'adult': False, 'gender': 2, 'id': 1344361, 'known_for_department': 'Acting', 'name': 'Kelvin Harrison Jr.', 'original_name': 'Kelvin Harrison Jr.', 'popularity': 14.917, 'profile_path': '/6kpDyaZzmSbqCNYuXZUfeMwS1bq.jpg', 'cast_id': 13, 'character': 'Taka (voice)', 'credit_id': '61290ee48d1b8e00274ef40e', 'order': 1}, {'adult': False, 'gender': 1, 'id': 1074676, 'known_for_department': 'Acting', 'name': 'Tiffany Boone', 'original_name': 'Tiffany Boone', 'popularity': 3.738, 'profile_path': '/9LwqRFdSzxVtnutDUg98YLq0bSz.jpg', 'cast_id': 33, 'character': 'Sarabi (voice)', 'credit_id': '662fc6c9072166012a6a405f', 'order': 2}, {'adult': False, 'gender': 2, 'id': 107529, 'known_for_d

In [81]:
# extract top 5 Actors/Actress from the movie
def cast_name(obj):
    name = []
    count = 0
    for i in ast.literal_eval(obj):
        name.append(i['name'])
        count +=1
        if count ==5:
            break
    return name

df['cast'] = df['cast'].apply(cast_name)

In [82]:
df['crew'].iloc[0]

"[{'adult': False, 'gender': 2, 'id': 42265, 'known_for_department': 'Production', 'name': 'Peter M. Tobyansen', 'original_name': 'Peter M. Tobyansen', 'popularity': 2.439, 'profile_path': None, 'credit_id': '605670771bf266002887f1e9', 'department': 'Production', 'job': 'Executive Producer'}, {'adult': False, 'gender': 1, 'id': 1681304, 'known_for_department': 'Editing', 'name': 'Joi McMillon', 'original_name': 'Joi McMillon', 'popularity': 0.119, 'profile_path': '/wdg9gRjHUQlBoyXILnQvfEOjl7b.jpg', 'credit_id': '605670a46f6a99003f35c9ee', 'department': 'Editing', 'job': 'Editor'}, {'adult': False, 'gender': 2, 'id': 5670, 'known_for_department': 'Art', 'name': 'Mark Friedberg', 'original_name': 'Mark Friedberg', 'popularity': 3.362, 'profile_path': '/3vs2xBKrXlXmkZEHBdhb0R5eeSJ.jpg', 'credit_id': '626847db924ce600504c771c', 'department': 'Art', 'job': 'Production Design'}, {'adult': False, 'gender': 2, 'id': 1179651, 'known_for_department': 'Acting', 'name': 'Lin-Manuel Miranda', 'orig

In [83]:
# Extracting director name of the movie
def director_name(obj):
    name = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            name.append(i['name'])
    return name

df['crew'] = df['crew'].apply(director_name)

In [84]:
df['keywords'].iloc[0]

"[{'id': 6054, 'name': 'friendship'}, {'id': 439, 'name': 'paradise'}, {'id': 2043, 'name': 'lion'}, {'id': 4344, 'name': 'musical'}, {'id': 9675, 'name': 'prequel'}, {'id': 10787, 'name': 'jungle'}, {'id': 171826, 'name': 'told in flashback'}, {'id': 171854, 'name': 'betrayal by friend'}, {'id': 177973, 'name': 'chance meeting'}, {'id': 245230, 'name': 'live action remake'}, {'id': 248927, 'name': 'found family'}, {'id': 262419, 'name': 'lost'}, {'id': 325778, 'name': 'bold'}, {'id': 325824, 'name': 'hopeful'}]"

In [85]:
# Extract main keywords of the movie
def keyword_name(obj):
    name = []
    for i in ast.literal_eval(obj):
        name.append(i['name'])
    return name
df['keywords'] = df['keywords'].apply(keyword_name)

In [86]:
df.head()

,id,title,overview,genre,cast,crew,keywords,image_url
0,762509,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...","[Adventure, Family, Animation]","[Aaron Pierre, Kelvin Harrison Jr., Tiffany Bo...",[Barry Jenkins],"[friendship, paradise, lion, musical, prequel,...",http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUut...
1,1241982,Moana 2,After receiving an unexpected call from her wa...,"[Animation, Adventure, Family, Comedy]","[Auliʻi Cravalho, Dwayne Johnson, Hualālai Chu...","[David G. Derrick Jr., Jason Hand, Dana Ledoux...","[sea, ocean, villain, musical, sequel, duringc...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
2,939243,Sonic the Hedgehog 3,"Sonic, Knuckles, and Tails reunite against a p...","[Action, Science Fiction, Comedy, Family]","[Jim Carrey, Ben Schwartz, Keanu Reeves, Idris...",[Jeff Fowler],"[moon, sequel, based on video game, burlesque,...",http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...
3,927342,Amaran,A heroic true story of Major Mukund Varadaraja...,"[Action, Drama, Adventure, War]","[Sivakarthikeyan, Sai Pallavi, Rahul Bose, Bhu...",[Rajkumar Periasamy],"[based on novel or book, affectation, based on...",http://image.tmdb.org/t/p/w500/gnsLzt6RdI4QINt...
4,822119,Captain America: Brave New World,After meeting with newly elected U.S. Presiden...,"[Action, Thriller, Science Fiction]","[Anthony Mackie, Harrison Ford, Danny Ramirez,...",[Julius Onah],"[hero, superhero, revenge, aftercreditsstinger...",http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...


In [87]:
# Text preprocessing
punctuations = string.punctuation
stop_words = stopwords.words('english')
ps = PorterStemmer()

def text_preprocessing(text):
    text = text.lower()
    text = word_tokenize(text)
    new_text = []
    
    for word in text:
        if word not in stop_words and word not in punctuations:
            new_text.append(word)
            
    y = []
    for word in new_text:
        y.append(ps.stem(word))
    return " ".join(y)

In [88]:
df['overview'] = df['overview'].apply(text_preprocessing)

In [89]:
# remove spaces between names because we are going to vectorize the text 
# if we won't remove spaces then chris of chris hemsworth and chris of chris evans will be taken as same words
def remove_space(obj):
    name = []
    for i in obj:
        name.append(i.replace(" ",""))
    return " ".join(name)

In [90]:
df['genre'] = df['genre'].apply(remove_space)
df['cast'] = df['cast'].apply(remove_space)
df['crew'] = df['crew'].apply(remove_space)
df['keywords'] = df['keywords'].apply(remove_space)

In [91]:
# create a new column consisting of only words
df['tags'] = df['overview'] + df['genre'] + df['cast'] + df['crew'] + df['keywords']

In [92]:
df.head()

,id,title,overview,genre,cast,crew,keywords,image_url,tags
0,762509,Mufasa: The Lion King,mufasa cub lost alon meet sympathet lion name ...,Adventure Family Animation,AaronPierre KelvinHarrisonJr. TiffanyBoone Kag...,BarryJenkins,friendship paradise lion musical prequel jungl...,http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUut...,mufasa cub lost alon meet sympathet lion name ...
1,1241982,Moana 2,receiv unexpect call wayfind ancestor moana jo...,Animation Adventure Family Comedy,AuliʻiCravalho DwayneJohnson HualālaiChung Ros...,DavidG.DerrickJr. JasonHand DanaLedouxMiller,sea ocean villain musical sequel duringcredits...,http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...,receiv unexpect call wayfind ancestor moana jo...
2,939243,Sonic the Hedgehog 3,sonic knuckl tail reunit power new adversari s...,Action ScienceFiction Comedy Family,JimCarrey BenSchwartz KeanuReeves IdrisElba Co...,JeffFowler,moon sequel basedonvideogame burlesque dualrol...,http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...,sonic knuckl tail reunit power new adversari s...
3,927342,Amaran,heroic true stori major mukund varadarajan ind...,Action Drama Adventure War,Sivakarthikeyan SaiPallavi RahulBose BhuvanAro...,RajkumarPeriasamy,basedonnovelorbook affectation basedontruestor...,http://image.tmdb.org/t/p/w500/gnsLzt6RdI4QINt...,heroic true stori major mukund varadarajan ind...
4,822119,Captain America: Brave New World,meet newli elect u.s. presid thaddeu ross sam ...,Action Thriller ScienceFiction,AnthonyMackie HarrisonFord DannyRamirez ShiraH...,JuliusOnah,hero superhero revenge aftercreditsstinger mar...,http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...,meet newli elect u.s. presid thaddeu ross sam ...


In [93]:
df2 = df.copy()
df2 = df2[['id','title','tags','image_url']]

In [94]:
df2.head()

,id,title,tags,image_url
0,762509,Mufasa: The Lion King,mufasa cub lost alon meet sympathet lion name ...,http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUut...
1,1241982,Moana 2,receiv unexpect call wayfind ancestor moana jo...,http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
2,939243,Sonic the Hedgehog 3,sonic knuckl tail reunit power new adversari s...,http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...
3,927342,Amaran,heroic true stori major mukund varadarajan ind...,http://image.tmdb.org/t/p/w500/gnsLzt6RdI4QINt...
4,822119,Captain America: Brave New World,meet newli elect u.s. presid thaddeu ross sam ...,http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...


In [95]:
# dropping duplicates changed the index hence reset it
df2.reset_index(inplace = True)

# Text Vectorization

In [96]:
cv = CountVectorizer(ngram_range = (1,3), max_features = 10000)
tfidf = TfidfVectorizer(ngram_range = (1,3), max_features = 10000)

tags_cv = cv.fit_transform(df2['tags']).toarray()
tags_tfidf = tfidf.fit_transform(df2['tags']).toarray()

In [97]:
tags_cv.shape, tags_tfidf.shape

((7780, 10000), (7780, 10000))

In [98]:
similarity_cv = cosine_similarity(tags_cv)
similarity_tfidf = cosine_similarity(tags_tfidf)

In [99]:
def recommend_cv(name):
    movie_index = df2[df2['title'] == name].index[0]
    recommend_movie = sorted(enumerate(similarity_cv[movie_index]),key = (lambda x: x[1]), reverse = True)[:6]
    for i in recommend_movie:
        print(df2['title'].iloc[i[0]])
        print(df2['image_url'].iloc[i[0]])
        
def recommend_tfidf(name):
    movie_index = df2[df2['title'] == name].index[0]
    recommend_movie = sorted(enumerate(similarity_cv[movie_index]),key = (lambda x: x[1]), reverse = True)[1:6]
    for i in recommend_movie:
        print(df2['title'].iloc[i[0]])
        print(df2['image_url'].iloc[i[0]])
        

In [100]:
recommend_cv('Harry Potter and the Philosopher\'s Stone')

Harry Potter and the Philosopher's Stone
http://image.tmdb.org/t/p/w500/wuMc08IPKEatf9rnMNXvIDxqP4W.jpg
Harry Potter and the Prisoner of Azkaban
http://image.tmdb.org/t/p/w500/aWxwnYoe8p2d2fcxOqtvAtJ72Rw.jpg
Harry Potter and the Goblet of Fire
http://image.tmdb.org/t/p/w500/fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg
Harry Potter and the Deathly Hallows: Part 2
http://image.tmdb.org/t/p/w500/c54HpQmuwXjHq2C9wmoACjxoom3.jpg
Harry Potter and the Half-Blood Prince
http://image.tmdb.org/t/p/w500/z7uo9zmQdQwU5ZJHFpv2Upl30i1.jpg
Harry Potter and the Chamber of Secrets
http://image.tmdb.org/t/p/w500/sdEOH0992YZ0QSxgXNIGLq1ToUi.jpg


In [101]:
recommend_tfidf('Harry Potter and the Philosopher\'s Stone')

Harry Potter and the Prisoner of Azkaban
http://image.tmdb.org/t/p/w500/aWxwnYoe8p2d2fcxOqtvAtJ72Rw.jpg
Harry Potter and the Goblet of Fire
http://image.tmdb.org/t/p/w500/fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg
Harry Potter and the Deathly Hallows: Part 2
http://image.tmdb.org/t/p/w500/c54HpQmuwXjHq2C9wmoACjxoom3.jpg
Harry Potter and the Half-Blood Prince
http://image.tmdb.org/t/p/w500/z7uo9zmQdQwU5ZJHFpv2Upl30i1.jpg
Harry Potter and the Chamber of Secrets
http://image.tmdb.org/t/p/w500/sdEOH0992YZ0QSxgXNIGLq1ToUi.jpg


In [102]:
recommend_cv('Avengers: Endgame')

Avengers: Endgame
http://image.tmdb.org/t/p/w500/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg
Avengers: Secret Wars
http://image.tmdb.org/t/p/w500/f0YBuh4hyiAheXhh4JnJWoKi9g5.jpg
Avengers: Doomsday
http://image.tmdb.org/t/p/w500/eO6OdA4RDRWeCVlDMcsoxWYFySD.jpg
Avengers: Age of Ultron
http://image.tmdb.org/t/p/w500/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg
The Avengers
http://image.tmdb.org/t/p/w500/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg
Avengers: Infinity War
http://image.tmdb.org/t/p/w500/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg


In [103]:
df2.drop(columns = 'index', inplace = True)

# both CountVectorizer and TfidfVectorizer gave same result so we only export cv
pickle.dump(similarity_cv,open('similarity.pkl','wb'))

# export df2
pickle.dump(df2,open('Movies Dataset.pkl','wb'))

# The exported Movies Dataset and similarity will be used by streamlit